In [ ]:
# pip install geopandas, geopy, opencage, ipywidgets (for tqdm)

In [3]:
import os
# Creating paths
in_folderpath = os.path.join("..", "in")

os.makedirs(in_folderpath, exist_ok=True)

In [4]:
import geopandas as gpd

# Loading the data into A geopandas dataframe
gdf = gpd.read_file(os.path.join(in_folderpath, "datacenters.geojson"))

# Filtering the dataframe to only include locations within the US
gdf_usa = gdf[gdf['country'] == 'USA']

print(len(gdf_usa))

DriverError: ..\in\datacenters.geojson: No such file or directory

# Opencage Geocoding

In [1]:
from opencage.geocoder import OpenCageGeocode
from shapely.geometry import Point
from tqdm import tqdm

In [2]:
api_key = '4176360a8df2453abab8cb3341a7be21'

geolocator = OpenCageGeocode(api_key)

# Iterate through each row to check and update geometry
for i, row in tqdm(gdf_usa.iterrows(), desc="Geocoding Addresses"):

    if row['geometry'] is None or row['geometry'].is_empty:
        
        # Creates an address for the geolocator
        address = f"{row['address']}, {row['postal']}, {row['city']}, {row['state']}, {row['country']}"
        
        # Returns a list of dictionaries with spatial information about the data center 
        location = geolocator.geocode(address)
        break
        if location:

            # Retrieves the geometry from the list of dictionaries
            geometry = location[0]['geometry']

            # Adds the retrived latitude and longditude to each row in data frame
            gdf_usa.at[i, 'geometry'] = Point(geometry['lng'], geometry['lat'])
            gdf_usa.at[i, 'latitude'] = geometry['lat']
            gdf_usa.at[i, 'longitude'] = geometry['lng']
        else:
            print("Geometry not found")


gdf_usa_filtered = gdf_usa[gdf_usa['geometry'].notna()]

print(len(gdf_usa_filtered))

csv_path = os.path.join(in_folderpath, "datacenters_usa.csv")

gdf_usa_filtered.to_csv(csv_path, index=False)

print("GeoJSON file has been formatted and saved.")

NameError: name 'OpenCageGeocode' is not defined

# Nominatim geocoder

In [1]:
from geopy.geocoders import Nominatim
from shapely.geometry import Point
import time
from tqdm import tqdm

In [33]:
email_address = "trine.rye@hotmail.com"

geolocator = Nominatim(user_agent=email_address)

# Iterate through each row to check and update geometry
for i, row in tqdm(gdf_usa.iterrows(), desc='Processing missing datacenter locations:'):

    # if a row (i.e. a datacenter) does not have a geometry (i.e. a location)
    if row['geometry'] is None or row['geometry'].is_empty:        
        # Create an address for the geolocator, by trying to combine the 
        address = f"{row['address']}, {row['postal']}, {row['city']}, {row['state']}, {row['country']}"
        location = geolocator.geocode(address)
        print(f'Primary address: {address}')
        print(f'geocode location: {location}')

        #we wait 2 seconds, as to not overload the nominatim API
        time.sleep(2)
        if location:
            # Primary location found, updating the geometry
            gdf_usa.at[i, 'geometry'] = Point(location.longitude, location.latitude)
            gdf_usa.at[i, 'latitude'] = location.latitude
            gdf_usa.at[i, 'longitude'] = location.longitude
        else:
            # If primary address does not return a location, try an alternate address
            alt_address = f"{row['postal']}, {row['city']}, {row['state']}, {row['country']}"
            print(f'Primary address didnt not return a geocoding location. Trying alternate address')
            print(f'Alternate address: {alt_address}')
            alt_location = geolocator.geocode(alt_address)
            print(f'alternate geocode location: {location}')
            
            #we wait 2 seconds, as to not overload the nominatim API
            time.sleep(2)
            if alt_location:
                # Alternate location found, updating the geometry
                gdf_usa.at[i, 'geometry'] = Point(alt_location.longitude, alt_location.latitude)
                gdf_usa.at[i, 'latitude'] = alt_location.latitude
                gdf_usa.at[i, 'longitude'] = alt_location.longitude
            else:
                # Neither primary nor alternate location found
                print(f"No geocoded locations found for {alt_address}")

gdf_usa_filtered = gdf_usa[gdf_usa['geometry'].notna()]

print(len(gdf_usa_filtered))

csv_path = os.path.join(in_folderpath, "datacenters_usa_2.csv")

gdf_usa_filtered.to_csv(csv_path, index=False)

print("GeoJSON file has been formatted and saved.")



Processing missing datacenter locations:: 0it [00:00, ?it/s]

Primary address: SECRET, within 2 mile from Interstate, 58103, Fargo, North Dakota, USA
geocode location: None
Primary address didnt not return a geocoding location. Trying alternate address
Alternate address: 58103, Fargo, North Dakota, USA
alternate geocode location: None


Processing missing datacenter locations:: 179it [00:09, 19.80it/s]


GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=412+E+Madison+St+suite+1010%2C+33602%2C+Tampa%2C+Florida%2C+USA&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

## References 

### Nominatim

https://medium.com/@gopesh3652/geocoding-with-python-using-nominatim-a-beginners-guide-220b250ca48d 

https://geopy.readthedocs.io/en/stable/ 
